# Try Fastai

Fastai provides an API for semantic segmentation. The first attempt is to learn how to use it.

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

from pathlib import Path
import os
import random

from fastai.callbacks.hooks import *
from fastai.utils.mem import *

## Preparation

Main challenge with fastai is the creation of databunch. After that its all straightforward.

In [ ]:
PROJ_DIR = Path(os.path.abspath("")).parent
PROJ_DIR

In [ ]:
path_train_img = PROJ_DIR / "data" / "raw" / "train_images"
path_train_labels = PROJ_DIR / "data" / "raw" / "train_labels"
path_test_img = PROJ_DIR / "data" / "raw" / "test_images"

In [ ]:
fnames = get_image_files(path_train_img)
fnames[:3]

In [ ]:
img_f = fnames[10]
img = open_image(img_f)
img.show(figsize=(5, 5))

In [ ]:
img = open_image(path_train_img / "0011165.jpg")
img.show(figsize=(5, 5))

### Reading Labels

In [ ]:
codes = ["Fish", "Flower", "Gravel", "Sugar"]
codes

## Create DataBunch

In [ ]:
free = gpu_mem_get_free_no_cache()
# the max size of bs depends on the available GPU RAM
if free > 8200:
    bs = 8
else:
    bs = 4
print(f"using bs={bs}, have {free}MB of GPU RAM free")

In [ ]:
bs=1

In [ ]:
get_y_fn = lambda x: path_train_labels / f"{x.stem}.png"

In [ ]:
src_size = np.array(img.shape[1:])
src_size

In [ ]:
size = src_size // 2
size

In [ ]:
data = (
    SegmentationItemList.from_folder(path_train_img)
    .split_by_rand_pct()
    .label_from_func(get_y_fn, classes=codes)
    .add_test_folder(path_test_img)
    .transform(get_transforms(), size=size, tfm_y=False)
    .databunch(bs=bs)
    .normalize(imagenet_stats)
)

data

In [ ]:
wd = 1e-2

In [ ]:
learn = unet_learner(data, models.resnet18, wd=wd)

In [ ]:
lr_find(learn)
learn.recorder.plot()